Користејќи го моделот FLAN-T5 со техниката zero-shot prompting за секој примерок 
од податочното множество за препознавање на навредлив текст одредете дали 
примерокот содржи навредлив текст или не.
Добиените предвидувања евалуирајте ги со метриките: точност
(accuracy_score), прецизност (precision_score), одзив (recall_score) и F1-
мерка (f1_score). Евалуацијата направете ја посебно за сите подмножества 
(подмножество за тренирање, валидација и тестирање).
Дали овој модел е подобар од моделите во втората лабораториска вежба?

In [1]:
# !pip install sentence_transformers

     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/86.0 kB ? eta -:--:--
     ------------------ ------------------- 41.0/86.0 kB 326.8 kB/s eta 0:00:01
     ------------------------------------ - 81.9/86.0 kB 508.4 kB/s eta 0:00:01
     -------------------------------------- 86.0/86.0 kB 482.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
     - ----------------------------------- 41.0/977.5 kB 960.0 kB/s eta 0:00:01
     --- ----------------------------------- 92.2/977.5 kB 1.0 MB/s eta 0:00:01
     ---- --------------------------------- 122.9/977.5 kB 1.2 MB/s eta 0:00:01
     ---- --------------------------------- 122.9/977.5 kB 1.2 MB/s eta 0:00:01
     --------- ---------------------------- 256.0/977.5 kB 1.1 MB/s eta 0:00:01
     ----------- -----------------------

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

In [4]:
test_en_path = "C:/Users/Mia/Desktop/FINKI/NLP/nlp/data/offensive text detection/test_en.txt"
train_en_path = "C:/Users/Mia/Desktop/FINKI/NLP/nlp/data/offensive text detection/train_en.txt"
val_en_path = "C:/Users/Mia/Desktop/FINKI/NLP/nlp/data/offensive text detection/val_en.txt"

In [10]:
train_en = pd.read_table(train_en_path).dropna()
test_en = pd.read_table(test_en_path).dropna()
val_en = pd.read_table(val_en_path).dropna()

In [11]:
train_en

,Sentence,Label
0,Pussy nobody asked for your input.,1
1,"Ok, this makes no sense. This will create vigi...",1
2,so fucking true. the amount of up and coming r...,1
3,Go f yourself Republican scum who put us here ...,1
4,Dumb fucking take. People want to do things.,1
...,...,...
3163,"This must be mostly bots but still, it's stran...",0
3164,Those actions are racist.,0
3165,Cultural Marxism isn't a thing.,0
3166,"LOL, anyone that questions the Democrat progra...",0


In [18]:
dataset = pd.concat([train_en, pd.concat([test_en, val_en])])

In [22]:
print(len(dataset))
print(len(train_en)+len(test_en)+len(val_en))

3962
3962


In [23]:
samples = dataset['Sentence'].values.tolist()
labels = dataset['Label'].values.tolist()

In [24]:
train_samples, test_samples, \
    train_labels, test_labels = train_test_split(samples, labels,
                                                     test_size=0.2, random_state=0,
                                                     stratify=labels)

In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

In [43]:
prompt_template = 'Classify the text into offensive or non-offensive, write them in 0 or 1 respectively: '

In [28]:
import re

In [48]:
y_pred = []
y_test = []
 # zero-shot
for sample, label in zip(test_samples, test_labels):
    prompt = f'{prompt_template}{sample}'
    
    input_data = tokenizer(prompt, return_tensors='pt')
    input_ids = input_data.input_ids
    
    output = model.generate(input_ids)
    pred_label = tokenizer.decode(output[0])
    
    pattern = re.compile('<.*?>')
    pred_label = int(re.sub(pattern, '', pred_label))

    if label == "non-offensive":
        label = 0
    else:
        label = 1
    y_test.append(label)
    y_pred.append(pred_label)
    print(f'Prompt: {prompt}\n'
          f'Sample: {sample}\n'
          f'True label: {label}\n'
          f'Predicted label: {pred_label}\n')


c:\Users\Mia\anaconda3\Lib\site-packages\transformers\generation\utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Prompt: Classify the text into offensive or non-offensive, write them in 0 or 1 respectively: Shhh. No logic is allowed in this sub. You’ll upset people here!
Sample: Shhh. No logic is allowed in this sub. You’ll upset people here!
True label: 1
Predicted label: 0

Prompt: Classify the text into offensive or non-offensive, write them in 0 or 1 respectively: They literally steal and sell babies in convenience stores. Sorry but most are undesirables. (a small % escape their culture and integrate)
Sample: They literally steal and sell babies in convenience stores. Sorry but most are undesirables. (a small % escape their culture and integrate)
True label: 1
Predicted label: 0

Prompt: Classify the text into offensive or non-offensive, write them in 0 or 1 respectively: No, you're just on the wrong side of history with your backwards beliefs and brainwashing.
Sample: No, you're just on the wrong side of history with your backwards beliefs and brainwashing.
True label: 1
Predicted label: 1



In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [49]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.626733921815889
Precision: 1.0
Recall: 0.626733921815889
F1 Score: 0.7705426356589147
